In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns

In [52]:
#1
tickets_headers = ['PUR_DATE', 'TRV_DATE', 'TKT_NUM', 'CPN_NUM', 'CABIN', 'AIRLINE', 'ORG', 'DST', \
    'FLT_NO', 'DEP_TIME', 'ARR_TIME', 'ARR_DATE', 'CPN_FARE']
tickets_date_headers = ['PUR_DATE', 'TRV_DATE', 'ARR_DATE']
tickets = pd.read_csv('tickets.txt', sep='\s+', lineterminator='\n', header=None, names=tickets_headers, parse_dates=tickets_date_headers)
tickets.drop_duplicates(inplace=True)
tickets.head()

,PUR_DATE,TRV_DATE,TKT_NUM,CPN_NUM,CABIN,AIRLINE,ORG,DST,FLT_NO,DEP_TIME,ARR_TIME,ARR_DATE,CPN_FARE
0,2016-02-01,2016-02-04,67637119886,1,Economy,DL,ATL,MCI,2075,1025,1132,2016-02-04,397
1,2016-02-01,2016-02-04,67637119886,2,Economy,DL,MCI,ATL,2256,1709,2012,2016-02-04,397
2,2016-02-01,2016-02-05,66163712203,1,Economy,DL,JFK,LAX,472,700,1040,2016-02-05,315
3,2016-02-01,2016-02-06,66163712203,2,Economy,DL,LAX,JFK,1262,2218,724,2016-02-07,315
4,2016-02-01,2016-02-11,11256761637,1,Economy,AA,ORD,PBI,1608,710,1107,2016-02-11,256


In [53]:
#1
schedule_date_headers = ['DEPT_DATE']
schedule = pd.read_csv('schedule.csv', header=0, parse_dates=schedule_date_headers)
schedule.drop_duplicates(inplace=True)
schedule.head()

,CARRIER,SHARED_AL,OP_INDICATOR,CODESHARE1,CODESHARE2,ORG,VIA,DST,SUM_GCD_MILE,FLIGHT_NO,STOPS,SEATS,DEPT_DATE,LOCAL_DEP_TIME
0,DL,NaN,O,NaN,NaN,DEN,NaN,DTW,1119,1410,0,160,2016-02-08,1628
1,DL,NaN,O,AF 2078,AZ 3264,JFK,NaN,PHX,2147,439,0,160,2016-02-08,1830
2,KL,DL,N,DL 1505,NaN,MSP,NaN,SFO,1585,7298,0,160,2016-02-08,1746
3,DL,OO,O,VS 3361,NaN,FSD,NaN,MSP,195,4556,0,50,2016-02-08,1710
4,CA,UA,N,UA 396,NaN,HNL,NaN,SFO,2394,7232,0,344,2016-02-08,2155


In [ ]:
#2
# The problem with the schedule data is a lot of duplicate values, whiche I eliminated them using drop_duplicates() method.
# regarding the tickets file, the file does not have a header row. So, I added them.abs
# generally, in both files, the date and time columns are not in standard format and need transformation. Either using parse_dates argument or with the use of to_datetime() method from pandas.

In [54]:
#3 Here is an overall descriptive table.
tickets['TRV_DATE'].describe()

count                     368
unique                    110
top       2016-02-29 00:00:00
freq                       13
first     2016-02-04 00:00:00
last      2016-10-03 00:00:00
Name: TRV_DATE, dtype: object

In [55]:
#4 the number of unique tickets is 214.
len(np.unique(tickets['TKT_NUM']))

214

In [56]:
#5 top 4 origins in terms of number of tickets are ATL, ATW, BNA, BOS
tickets.groupby('ORG').count()
tickets['ORG_CNT'] = tickets['ORG']
result = tickets.groupby('ORG', as_index = False)['ORG_CNT'].count()
result.sort_values(by='ORG_CNT', ascending=False)
result['ORG'][:4]


0    ATL
1    ATW
2    BNA
3    BOS
Name: ORG, dtype: object

In [73]:
#6

# let's do an inner join of the two dataframes first
merged = pd.merge(left=schedule, right=tickets, left_on=['CARRIER', 'FLIGHT_NO', 'DEPT_DATE'], right_on=['AIRLINE', 'FLT_NO', 'TRV_DATE'])
merged.head()

# now we have all data for calculating Yield in one merged dataframe
merged['YIELD'] = merged['CPN_FARE'] / merged['SUM_GCD_MILE']
merged.head()

# the average yield for the three classes is:
result = merged.groupby('CABIN', as_index = False)['YIELD'].mean()
result

,CABIN,YIELD
0,Business,0.526245
1,Economy,0.237652
2,First,0.675398


In [74]:
#7

# first, let's subset the first coupons
first_coupons = tickets[tickets['CPN_NUM'] == 1]

# now, we could calculate lead time:
first_coupons['LEAD_TIME']= first_coupons['TRV_DATE'] - first_coupons['PUR_DATE']
mean = np.mean(first_coupons['LEAD_TIME'])
maximum = first_coupons['LEAD_TIME'].max()
minimum = first_coupons['LEAD_TIME'].min()
print(f'mean of lead time is : {mean}')
print(f'maximum of lead time is : {maximum}')
print(f'minimum of lead time is : {minimum}')

mean of lead time is : 37 days 17:22:59.439252336
maximum of lead time is : 220 days 00:00:00
minimum of lead time is : 0 days 00:00:00


In [75]:
#8

tmp = {str(k): f.to_numpy().tolist() for k, f in tickets.groupby('TKT_NUM')}
one_way_cnt, return_cnt = 0, 0
for ticket_no, legs in tmp.items():
    if legs[0][6] == legs[-1][7]:
        return_cnt +=1
    else:
        one_way_cnt += 1
print(f'Return: {return_cnt}')
print(f'One-Way: {one_way_cnt}')
print(f'Total Tickets: {len(tmp)}')

Return: 87
One-Way: 127
Total Tickets: 214


In [89]:
#9

# drop rows with at least one NaN value
merged_non_null = merged[['CODESHARE1', 'CODESHARE2']].dropna(thresh=1)
print(f'Number of tickets sold on codeshared flights is: {len(merged_non_null)}')

Number of tickets sold on codeshared flights is: 336


,CODESHARE1,CODESHARE2
124,AF 8668,KL 6706
125,AF 8668,KL 6706
126,AY 4096,BA 1578
127,QF 4506,NaN
128,AS 1005,BA 4790
...,...,...
373,BA 4795,GF 6882
374,VA 6596,WS 6570
375,CM 2194,NZ 9285
376,AM 5908,VS 2529
